In [1]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 65kB/s 
     |████████████████████████████████| 204kB 40.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=a7f33601d812fe3dbd210f462b31cbc453b7d343b14ba2cf4e33add45dfce356
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import sys
import re
from pyspark import SparkContext, SparkConf
import numpy as np
import time as time
from operator import itemgetter

In [2]:
# create Spark context with necessary configuration
sc = SparkContext("local","PySpark HITS algorithm")

In [3]:
filepath = '/content/drive/My Drive/Coursework/CS 5683_Big_Data/Assignment-6/email-Eu-core.txt'
links = sc.textFile(filepath)

#links.repartition(32)

#transpose
links_formatted = links.map(lambda x: (int(x.split(' ')[0]),int(x.split(' ')[1])))
links_formatted_t = links.map(lambda x: (int(x.split(' ')[1]),int(x.split(' ')[0])))

print(links_formatted.count())
print(links_formatted.take(3))


25571
[(0, 1), (2, 3), (2, 4)]


In [4]:
links_formatted_list = links_formatted.map(lambda x: (x[0],[x[1]]))
link_formatted_dict = links_formatted_list.reduceByKey(lambda a,b: a+b)

#transpose
links_formatted_list_t = links_formatted_t.map(lambda x: (x[0],[x[1]]))
link_formatted_dict_t = links_formatted_list_t.reduceByKey(lambda a,b: a+b)

print('The number of nodes in L is ',link_formatted_dict.count())
print('The number of nodes in L.T is ',link_formatted_dict_t.count())


The number of nodes in L is  868
The number of nodes in L.T is  991


In [5]:
hubs = links_formatted.flatMap(lambda x: x).distinct().map(lambda x: (x, 1.0))
n_nodes = hubs.count()

print('Total number of nodes is ', n_nodes)
elements = list(np.zeros(n_nodes))
rdd = sc.parallelize(elements)

rdd = rdd.zipWithIndex()

def getList(x):
  return_list = list(np.zeros(n_nodes))
  return return_list

matRdd = rdd.map(lambda x: getList(x[0]))

#transpose
matRdd_t = rdd.map(lambda x: getList(x[0]))


Total number of nodes is  1005


In [6]:
def getList_empty(x):
  return_list = []
  return return_list

empty_rdd = rdd.map(lambda x: getList_empty(x[0]))
empty_rdd = empty_rdd.zipWithIndex()
empty_rdd = empty_rdd.map(lambda x: (x[1], x[0]))

def add_two_lists(x):
  if (x[1][0]) == None :
    return (x[0],['A'])
  else:
    return (x[0],x[1][0])

zip_empty = link_formatted_dict.rightOuterJoin(empty_rdd).map(lambda x: add_two_lists(x))
zip_empty_p = zip_empty.repartition(1)

#transpose
zip_empty_t = link_formatted_dict_t.rightOuterJoin(empty_rdd).map(lambda x: add_two_lists(x))
zip_empty_p_t = zip_empty_t.repartition(1)


In [7]:
for a in zip_empty_t.collect(): 
  if a[0] == 78:
    print(a)

(78, [77, 497, 539, 310, 114, 62, 340, 419, 531, 482, 160, 79, 548, 366, 424, 93, 82, 13, 420, 121, 581, 107, 405])


In [9]:
matRdd = matRdd.zipWithIndex()
matRdd = matRdd.map(lambda x: (x[1], x[0]))

#transpose
matRdd_t = matRdd_t.zipWithIndex()
matRdd_t = matRdd_t.map(lambda x: (x[1], x[0]))

sampleWordsRDD6 = zip_empty_p.join(matRdd)
print(sampleWordsRDD6.count())

#transpose
sampleWordsRDD6_t = zip_empty_p_t.join(matRdd_t)
print(sampleWordsRDD6_t.count())


1005
1005


In [12]:
def form_matrix(x):
  if x[1][0][0] == 'A':
    l1 = [0 for i in range(n_nodes)]
    return (x[0],l1)
  else:
    for k in x[1][0]:
      x[1][1][1][k] = 1
    return (x[0],x[1][1][1])

sampleWordsRDD7 = sampleWordsRDD6.map(lambda x: form_matrix(x))

#transpose
sampleWordsRDD7_t = sampleWordsRDD6_t.map(lambda x: form_matrix(x))


In [13]:
hubs_ones = [1 for i in range(n_nodes)]
hubs = sc.parallelize(hubs_ones)
hubs_updated = hubs_ones

def product_sum(x):
  if x[1][1] == None:
    return 0
  else:
    # return (x[0],np.sum(np.array(x[1]*np.array(x[2]))))
    return np.sum(np.array(x[1]*np.array(x[2])))

import time as time

for i in range(40):
  start = time.time()

  matrix_hubs = sampleWordsRDD7.map(lambda x: (x[0],x[1],hubs_updated))
  matrix_product_hubs = matrix_hubs.map(lambda x: product_sum(x))

  auths_list = matrix_product_hubs.collect()
  auths_list = [auths_list[i]/max(auths_list) for i in range(len(auths_list))]

  matrix_t_auths = sampleWordsRDD7_t.map(lambda x: (x[0],x[1],auths_list))
  matrix_t_product_auths = matrix_t_auths.map(lambda x: product_sum(x))

  hubs_updated = matrix_t_product_auths.collect()
  hubs_updated = [hubs_updated[i]/max(hubs_updated) for i in range(len(hubs_updated))]

  print(i, time.time() - start)

Py4JJavaError: ignored

In [ ]:
tup_li = []
for i in range(0, 1005):
  t = (i, hubs_updated[i])
  tup_li.append(t)

from operator import itemgetter
sorte_tuple_list_asc = sorted(tup_li,key=itemgetter(1), reverse=False)
sorte_tuple_list_dsc = sorted(tup_li,key=itemgetter(1), reverse=True)

print(sorte_tuple_list_asc)
print(sorte_tuple_list_dsc)

[(289, 0.0), (407, 0.0), (464, 0.0), (488, 0.0), (491, 0.0), (497, 0.0), (498, 0.0), (499, 0.0), (500, 0.0), (501, 0.0), (502, 0.0), (716, 0.0), (765, 0.0), (842, 0.0), (943, 0.0), (981, 0.0), (982, 0.0), (984, 0.0), (997, 0.0), (998, 0.0), (999, 0.0), (1000, 0.0), (1001, 0.0), (1002, 0.0), (1003, 0.0), (1004, 0.0), (892, 6.321465493915794e-06), (477, 3.2768615685955766e-05), (435, 5.176631282221961e-05), (404, 7.814948092565474e-05), (389, 8.381747143366979e-05), (205, 0.0002015453456180228), (473, 0.00023224156492007258), (977, 0.0002409939530677467), (399, 0.00024491315705434764), (377, 0.00026164953022897176), (434, 0.00030248421671855434), (942, 0.0003193004356872582), (994, 0.00033209849390666916), (396, 0.000489241625747625), (987, 0.0005269457842681607), (817, 0.0005293026890478701), (376, 0.0005372485864054237), (424, 0.0005492389588427586), (299, 0.0006360980204368576), (873, 0.0006477041838992945), (945, 0.0006888464592877815), (452, 0.0007423247841703567), (709, 0.000742324